In [375]:
import numpy as np
import tensorflow as tf

In [376]:
%run 11-single_embedding_transformer.ipynb

18887
18887
6
(55698, 79)
(55698, 79)
(64, 79, 1024)
(64, 79, 1024)
(64, 79)
(64, 79, 18887)


In [377]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [378]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [379]:
def masked_loss(label, pred):
  print(label)
  mask = label != 0
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss


def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 0

  match = match & mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [380]:
transformer.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy])

In [384]:
transformer.fit(batched_dataset,
                epochs=1)

435/435 [==============================] - 2372s 5s/step - loss: 0.5662 - accuracy: 0.9207


In [563]:
starting_string = "this is the princess from the"

In [564]:
used_tokenizer = data_object.tokenizer

In [565]:
encoded_input = used_tokenizer.texts_to_sequences([starting_string])[0]
encoded_input

[18, 11, 1, 866, 44, 1]

In [566]:
inner_token_list = [encoded_input]
output_generated = ""
iter_thing = 1

In [567]:
inner_input = tf.convert_to_tensor(inner_token_list) 
inner_input

<tf.Tensor: shape=(1, 6), dtype=int32, numpy=array([[ 18,  11,   1, 866,  44,   1]])>

In [571]:

# Get the token to word mapping
token_to_word_mapping = used_tokenizer.index_word

# Print the mapping
print("Token to Word Mapping:")
for token, word in token_to_word_mapping.items():
    print(f"{token}: {word}")

Token to Word Mapping:
1: the
2: and
3: to
4: i
5: of
6: a
7: you
8: my
9: in
10: that
11: is
12: not
13: it
14: with
15: his
16: for
17: me
18: this
19: be
20: your
21: he
22: but
23: have
24: as
25: him
26: thou
27: so
28: will
29: what
30: we
31: all
32: no
33: thy
34: by
35: shall
36: our
37: are
38: if
39: do
40: her
41: on
42: thee
43: lord
44: from
45: now
46: good
47: they
48: at
49: was
50: or
51: enter
52: which
53: more
54: their
55: sir
56: come
57: would
58: then
59: king
60: well
61: them
62: am
63: o
64: let
65: when
66: how
67: us
68: here
69: than
70: like
71: hath
72: did
73: man
74: an
75: upon
76: know
77: there
78: one
79: i'll
80: make
81: may
82: love
83: she
84: were
85: should
86: say
87: yet
88: go
89: out
90: must
91: had
92: such
93: 'tis
94: these
95: why
96: see
97: give
98: where
99: most
100: too
101: take
102: speak
103: who
104: some
105: can
106: up
107: exeunt
108: mine
109: time
110: great
111: tell
112: never
113: much
114: heart
115: think
116: '


In [570]:
for i in range(1):
    inner_input = tf.convert_to_tensor([pad_list(inner_token_list[0], 80)])
    predictions  = transformer(inner_input)[:, iter_thing:, :]
    predicted_id = tf.argmax(predictions, axis=-1)
    token = predicted_id.numpy()[0][0]
    inner_token_list[0].append(token)

    if token ==0:
        output_generated += "\n"
    else:
        text = used_tokenizer.index_word[token]
        output_generated += (' ' + text)
        
    iter_thing += 1
    #inner_token_list[0].append(token)

print(output_generated)



 the princess



In [558]:
predictions

<tf.Tensor: shape=(1, 77, 18887), dtype=float32, numpy=
array([[[ 5.331877  ,  3.0769868 ,  4.11668   , ..., -6.9925117 ,
         -7.0183353 , -7.291755  ],
        [ 2.9410036 ,  0.3127428 ,  1.6045855 , ..., -4.05875   ,
         -4.168683  , -4.299876  ],
        [ 6.0158896 ,  2.9027834 ,  3.778853  , ..., -7.3943267 ,
         -7.439003  , -7.6990952 ],
        ...,
        [11.023119  , -1.4153713 , -0.4864066 , ..., -9.081871  ,
         -9.271184  , -9.281945  ],
        [11.023244  , -1.4121944 , -0.48928687, ..., -9.081889  ,
         -9.2710285 , -9.283908  ],
        [11.022485  , -1.4097282 , -0.49344507, ..., -9.082061  ,
         -9.270533  , -9.284898  ]]], dtype=float32)>

In [300]:
transformer.save_weights('./transformer_v2_80word/transformer_v2_80word')

In [143]:
#inner_input = tf.transpose(working_input.stack())
inner_input = tf.convert_to_tensor(inner_token_list) 

predictions  = transformer(inner_input)[:, -1:, :]
predicted_id = tf.argmax(predictions, axis=-1)
token = predicted_id.numpy()[0][0]
text = used_tokenizer.index_word[token]
#working_input.concat(3)

#generated_o+=text
#output_array = working_input.write(my_iter_obj, predicted_id[0])

#my_iter_obj+=1

print(text)
print(predicted_id[0])

InvalidArgumentError: Exception encountered when calling layer 'positional_embedding_15' (type PositionalEmbedding).

{{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:CPU:0}} slice index 1 of dimension 0 out of bounds. [Op:StridedSlice] name: transformer_4/decoder_9/positional_embedding_15/strided_slice/

Call arguments received by layer 'positional_embedding_15' (type PositionalEmbedding):
  • x=tf.Tensor(shape=(3,), dtype=int32)

In [286]:
for batch in batched_dataset:
    print(batch[0][0])
    break

tf.Tensor(
[  87  248  143   34  962  145  609  127   16  106   35  106  106   87
  248    7   41   35 1228  351    3  192   65    3 3678   35 1228 1228
   87  248   87    7   92 1052  214   11 2085  544    3    1  281   35
   34 2333   34 2333   87  248   67   78  474   26    2  369   23 1308
   53   37  162 2086  646    8 3679   35   33   50 4648 1013   67   15
   18  167  154  154  149  248   72  226   46], shape=(79,), dtype=int32)


In [130]:
tensor_encoded_input = input_to_tensor = tf.convert_to_tensor(encoded_input) 

In [131]:
tensor_encoded_input

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 104,   20, 1301])>

In [132]:
generated_o = ""

In [ ]:
#inner_input = tf.transpose(working_input.stack())


predictions  = transformer(inner_input)[:, -1:, :]
predicted_id = tf.argmax(predictions, axis=-1)
token = predicted_id.numpy()[0][0]
text = used_tokenizer.index_word[token]
working_input.concat(3)

generated_o+=text
output_array = working_input.write(my_iter_obj, predicted_id[0])

my_iter_obj+=1

print(text)
print(predicted_id[0])

In [134]:
working_input = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
working_input.write(0,tf.cast(tensor_encoded_input, dtype=tf.int64))

In [135]:
my_iter_obj = 1

In [136]:
print(generated_o)
print(working_input)

In [137]:
inner_input = working_input.stack()#tf.transpose(working_input.stack())
inner_input

<tf.Tensor: shape=(1, 3), dtype=int64, numpy=array([[ 104,   20, 1301]], dtype=int64)>

In [140]:
#inner_input = tf.transpose(working_input.stack())


predictions  = transformer(inner_input)[:, -1:, :]
predicted_id = tf.argmax(predictions, axis=-1)
token = predicted_id.numpy()[0][0]
text = used_tokenizer.index_word[token]
working_input.concat(3)

generated_o+=text
output_array = working_input.write(my_iter_obj, predicted_id[0])

my_iter_obj+=1

print(text)
print(predicted_id[0])

ValueError: Incompatible shape for value ((1,)), expected ((3,))

In [73]:
predictions  = transformer(tensor_encoded_input)[:, -1:, :]
predictions

<tf.Tensor: shape=(1, 1, 11913), dtype=float32, numpy=
array([[[-13.653607 ,   3.5102906,   4.55223  , ..., -13.595964 ,
         -13.772838 , -13.836386 ]]], dtype=float32)>

In [72]:
predicted_id = tf.argmax(predictions, axis=-1)
predicted_id

<tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[3]], dtype=int64)>

3

In [84]:
text = used_tokenizer.index_word[predicted_id.numpy()]
#.lookup(predicted_id)[0]

TypeError: unhashable type: 'numpy.ndarray'

In [82]:
text[3]

'to'